# Prompt Template (Prompt Engineering)
> 프롬프트는 모델에 대한 입력을 의미

- 프롬프트 템플릿은 이 입력의 구성을 담당한다.
- LangChain은 프롬프트를 쉽게 구성하고 작업할 수 있도록 여러 클래스와 함수를 제공한다.
- 반복적인 프롬프트를 삽입해야하는 경우, Prompt Template를 통해 간편하게 LLM을 활용할 수 있다.

1. `PromptTemplate` :
일반적인 프롬프트 템플릿을 생성할 때 활용한다.
2. `ChatPromptTemplate` :
채팅 LLM에 프롬프트를 전달하는데 황용할 수 있는 특화된 프롬프트 템플릿이다.


In [ ]:
!pip install openai langchain langchain-openai

In [13]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPEN_AI_API')

In [55]:
from langchain.prompts import PromptTemplate

string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")
answer = string_prompt.format(subject="soccer")
answer

'tell me a joke about soccer'

In [58]:
from langchain.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
answer = chat_prompt.format(subject="soccer")
print(answer)

Human: tell me a joke about soccer


## 1.PromptTemplate

In [59]:
from langchain.prompts import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
{재료}

"""

prompt_template = PromptTemplate(
    input_variables = ['재료'],
    template = template
)

prompt_template

PromptTemplate(input_variables=['재료'], template='\n너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.\n내가 가진 재료는 아래와 같아.\n\n<재료>\n{재료}\n\n')

In [60]:
print(prompt_template.format(재료 = '양파, 계란, 사과, 빵'))


너는 요리사야. 내가 가진 재료들을 갖고 만들 수 있는 요리를 추천하고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아.

<재료>
양파, 계란, 사과, 빵




In [61]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain

model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=512)
chain = LLMChain(llm=model, prompt=prompt_template)
answer = chain.run(재료 = '양파, 계란, 사과, 빵')

print(answer)

가지고 계신 재료로 추천해드릴 요리는 "양파 계란 볶음"입니다.
아래는 그 요리의 레시피입니다.

[양파 계란 볶음 레시피]
1. 양파를 깍두기 형태로 썰어줍니다.
2. 팬에 식용유를 두르고 양파를 넣어 중간 불에서 볶아줍니다.
3. 양파가 투명해질 때까지 볶은 후 계란을 풀어 팬에 넣어줍니다.
4. 계란이 익을 때까지 섞어가며 볶아줍니다.
5. 양파와 계란이 골고루 섞이고 익게 되면 소금과 후추로 간을 해줍니다.
6. 요리가 완성되면 접시에 담아내어 사과와 빵과 함께 즐기면 됩니다.

감사합니다. 즐거운 요리 시간 되세요!


## 2.ChatPromptTemplate

In [71]:
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)

from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [79]:
# ChatGPT 모델 로드
chatgpt = ChatOpenAI(temperature=0)

# ChatGPT 역할 부여
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

# 사용자가 입력할 매개변수 템플릿 선언
human_template = "{재료}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# ChatPromptTemplate 설정
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
answer = chatgpt(chat_prompt.format_prompt(재료="양파, 계란, 사과, 빵").to_messages())
print(answer.content)

가지고 있는 재료로 만들 수 있는 요리 중 하나는 "양파 계란말이"입니다. 이 요리는 양파와 계란을 함께 볶아서 만드는 간단하면서도 맛있는 요리입니다. 아래는 양파 계란말이의 레시피입니다.

[양파 계란말이 레시피]
1. 양파를 깍두기 모양으로 썰어줍니다.
2. 팬에 식용유를 두르고 양파를 넣어 중간 불에서 볶아줍니다.
3. 양파가 투명해질 때까지 볶은 후 계란을 풀어 넣어줍니다.
4. 계란이 익을 때까지 저어가며 볶아줍니다.
5. 양파 계란말이가 완성되면 빵과 함께 즐겨주세요.

이렇게 간단하게 양파 계란말이를 만들 수 있습니다. 재료들을 활용하여 맛있는 요리를 즐기세요!


## 3.Few-shot Prompt Template
- Few-shot은 딥러닝 모델이 결과물을 출력할 때 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론이다.
- LLM 역시 Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물을 출력한다.
- 원하는 결과물의 형태가 특수하거나 구조화된 답변을 원할 경우 결과물의 예시를 몇가지 제시하여 **결과물의 품질을 향상**시킬 수 있다.

In [62]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
  {
    "question": "아이유로 삼행시 만들어줘",
    "answer":
"""
아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.
"""
  },
  {
    "question": "김민수로 삼행시 만들어줘",
    "answer":
"""
김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!
"""
  }
]

In [63]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\n{answer}")

print(example_prompt.format(**examples[0]))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.



In [64]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

print(prompt.format(input="호날두로 삼행시 만들어줘"))

Question: 아이유로 삼행시 만들어줘

아: 아이유는
이: 이런 강의를 들을 이
유: 유가 없다.


Question: 김민수로 삼행시 만들어줘

김: 김치는 맛있다
민: 민달팽이도 좋아하는 김치!
수: 수억을 줘도 김치는 내꺼!


Question: 호날두로 삼행시 만들어줘


In [66]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=512)
chain = LLMChain(llm=model, prompt=prompt)

print(">>> PromptTemplate 미적용")
print(model.invoke("호날두로 삼행시 만들어줘"))

print(">>> PromptTemplate 적용")
print(model.invoke(prompt.format(input="호날두로 삼행시 만들어줘")))

>>> PromptTemplate 미적용
content='호날두, 축구왕\n공을 다루는 솜씨 정말 대단해\n파리에서 빛나는 네 모습을 보면 황홀해'
>>> PromptTemplate 적용
content='호: 호날두는\n날: 날개를 펼치는\n두: 두 발로 뛰어넘는다.'


## 4.Example Selector
- Few-shot 예제를 동적으로 입력하고 싶은 경우, Example Selector를 활용할 수 있다.
- LLM이 여러 작업을 수행하도록 만들되 원하는 범위의 대답을 출력하도록 하려면 사용자의 입력에 동적으로 반응해야 한다.
- 이와 동시에 예제를 모두 학습시키는 것이 아니라 적절한 예실만 포함하도록 함으로써 입력 프롬프트의 길이를 제한하고, 이를 통해 오류가 발생하지 않도록 조절할 수 있다.

In [82]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector # 사용자 입력과 유사한 예제를 반환하는 함수
from langchain.vectorstores import Chroma # VectorStore 통해서 임베딩된 값들을 비교
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [ ]:
!pip install chromadb
!pip install tiktoken

In [94]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    OpenAIEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=1
)

similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

In [95]:
print(similar_prompt.format(단어="무서운"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 불안
Output: 안정

Input: 무서운
Output:


In [96]:
print(similar_prompt.format(단어="큰 비행기"))

주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘

Input: 긴 기차
Output: 짧은 기차

Input: 큰 비행기
Output:


In [97]:
query = "큰 비행기"

print(model.invoke(similar_prompt.format(단어=query)))

content='작은 비행기'


## 5.Output Parser
- LLM의 **답변을 원하는 형태로 고정**할 때 OutputParser 함수를 활용한다.
- JSON, List 등의 다양한 형식의 답변을 고정하여 출력할 수 있다.
- 서비스와 결합할 때 유용하게 활용될 수 있다.

In [117]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [118]:
prompt = PromptTemplate(
    template="{주제} 5개를 추천해줘.\n{format_instructions}",
    input_variables=["주제"],
    partial_variables={"format_instructions": format_instructions}
)

In [119]:
model = OpenAI(temperature=0)
print(model.model_name)

gpt-3.5-turbo-instruct


In [129]:
_input = prompt.format(주제="영화")
output = model.invoke(_input)
print(output)



1. Parasite
2. The Shawshank Redemption
3. Inception
4. The Godfather
5. Pulp Fiction


In [123]:
output_parser.parse(output)

['1. Parasite\n2. The Shawshank Redemption\n3. Inception\n4. The Godfather\n5. Pulp Fiction']